# Overview of this notebook

In this, we explore the notion of agentic RAG using DeepSeek.

Agentic RAG is an RAG that integrates the capabilities of Agent, and the core capabilities of Agent are autonomous reasoning and action.

Therefore, Agentic RAG brings the autonomous planning capabilities of AI agents (such as routing, action steps, reflection, etc.) into traditional RAG to adapt to more complex RAG query tasks.

Agentic RAG's **“agent”** feature is mainly reflected in the retrieval stage. Compared to the retrieval process in traditional RAG, Agentic RAG is more capable of:

- Deciding whether to search (autonomous decision-making)
- Choosing which search engine to use (autonomous planning)
- Evaluating the retrieved context and deciding whether to re-search (self-planning)
- Determining whether to use external tools

In [ ]:
!pip install streamlit chromadb

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict, Annotated
from typing import Sequence
from langchain_openai import OpenAIEmbeddings
import re
import os
import streamlit as st
import requests
from langchain.tools.retriever import create_retriever_tool
import chromadb

In [4]:
from dotenv import load_dotenv
load_dotenv('.env')

True

In [5]:
openai_api_key = os.getenv("OPENAI_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

In [6]:
# Create Dummy Data
research_texts = [
    "Research Report: Results of a New AI Model Improving Image Recognition Accuracy to 98%",
    "Academic Paper Summary: Why Transformers Became the Mainstream Architecture in Natural Language Processing",
    "Latest Trends in Machine Learning Methods Using Quantum Computing"
]

development_texts = [
    "Project A: UI Design Completed, API Integration in Progress",
    "Project B: Testing New Feature X, Bug Fixes Needed",
    "Product Y: In the Performance Optimization Stage Before Release"
]

Let’s process the data by splitting it into smaller parts, converting it into document objects, and then creating vector embeddings.

In [7]:
# Create a retriever of type BaseRetriever, from the dummy data
def create_retriever(texts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
    docs = text_splitter.create_documents(texts)
    return Chroma.from_documents(docs, OpenAIEmbeddings()).as_retriever()

research_retriever = create_retriever(research_texts)
development_retriever = create_retriever(development_texts)

In [8]:
research_tool = create_retriever_tool(
    research_retriever,  # Retriever object
    "research_db_tool",  # Name of the tool to create
    "Search information from the research database."  # Description of the tool
)

development_tool = create_retriever_tool(
    development_retriever,
    "development_db_tool",
    "Search information from the development database."
)

# Combine the created research and development tools into a list
tools = [research_tool, development_tool]

The documents retrieved by this tool can be extracted from the return value of the function that this tool wraps

In [25]:
# This will be used in multiple steps of the graph
def call_deepseek(prompt: str):
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {deepseek_api_key}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "deepseek-chat",
        "messages": [{
            "role": "user",
            "content": prompt
        }],
        "temperature": 0.7,
        "max_tokens": 1024
    }
    
    response = requests.post(
        "https://api.deepseek.com/v1/chat/completions",
        headers=headers,
        json=data,
        verify=False
    )

    return response

Now, we can implement an agent as a router for queries. Using a prompt, we can categorize the query as research or development related. We can connect to DeepSeek with temperature of 0.7 for balanced responses, and when the API responds, check if it’s a research or development query, then use the appropriate retriever to find relevant documents. If it doesnt fit either query, return a direct answer. This agent can be thought of as a traffic controller, to route to the right data source for queries.

In [26]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[AIMessage|HumanMessage|ToolMessage], add_messages]

def traffice_router_agent(state: AgentState):
    print("---CALL AGENT---")
    messages = state["messages"]

    if isinstance(messages[0], tuple):
        user_message = messages[0][1]
    else:
        user_message = messages[0].content

    # Structure prompt for consistent text output
    prompt = f"""Given this user question: "{user_message}"
    If it's about research or academic topics, respond EXACTLY in this format:
    SEARCH_RESEARCH: <search terms>
    
    If it's about development status, respond EXACTLY in this format:
    SEARCH_DEV: <search terms>
    
    Otherwise, just answer directly.
    """

    response = call_deepseek(prompt)
    
    if response.status_code == 200:
        response_text = response.json()['choices'][0]['message']['content']
        print("Raw response:", response_text)
        
        # Format the response into expected tool format
        if "SEARCH_RESEARCH:" in response_text:
            query = response_text.split("SEARCH_RESEARCH:")[1].strip()
            # Use direct call to research retriever
            results = research_retriever.invoke(query)
            return {"messages": [AIMessage(content=f'Action: research_db_tool\n{{"query": "{query}"}}\n\nResults: {str(results)}')]}
        elif "SEARCH_DEV:" in response_text:
            query = response_text.split("SEARCH_DEV:")[1].strip()
            # Use direct call to development retriever
            results = development_retriever.invoke(query)
            return {"messages": [AIMessage(content=f'Action: development_db_tool\n{{"query": "{query}"}}\n\nResults: {str(results)}')]}
        else:
            return {"messages": [AIMessage(content=response_text)]}
    else:
        raise Exception(f"API call failed: {response.text}")

Now for a grading function, which will check if there are documents in the result. If there are, move forward to an answer. If not, suggest rewriting the query for better results.

In [27]:
def simple_grade_documents(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    print("Evaluating message:", last_message.content)
    
    # Check if the content contains retrieved documents
    if "Results: [Document" in last_message.content:
        print("---DOCS FOUND, GO TO GENERATE---")
        return "generate"
    else:
        print("---NO DOCS FOUND, TRY REWRITE---")
        return "rewrite"

In [28]:
def rewrite(state: AgentState):
    print("---REWRITE QUESTION---")
    messages = state["messages"]
    original_question = messages[0].content if len(messages)>0 else "N/A"

    response = call_deepseek(f"Rewrite this question to be more specific and clearer: {original_question}")

    if response.status_code == 200:
        response_text = response.json()['choices'][0]['message']['content']
        print("Rewritten question:", response_text)
        return {"messages": [AIMessage(content=response_text)]}
    else:
        raise Exception(f"API call failed: {response.text}")

Now, the generate final answer function. We can use DeepSeek to generate the final answer for us based on the found documents.

In [29]:
def generate(state: AgentState):
    print("---GENERATE FINAL ANSWER---")
    messages = state["messages"]
    question = messages[0].content if isinstance(messages[0], tuple) else messages[0].content
    last_message = messages[-1]

    # Extract the document content from the results
    docs = ""
    if "Results: [" in last_message.content:
        results_start = last_message.content.find("Results: [")
        docs = last_message.content[results_start:]
    print("Documents found:", docs)

    
    prompt = f"""Based on these research documents, summarize the latest advancements in AI:
    Question: {question}
    Documents: {docs}
    Focus on extracting and synthesizing the key findings from the research papers.
    """

    response = call_deepseek(prompt)
    
    if response.status_code == 200:
        response_text = response.json()['choices'][0]['message']['content']
        print("Final Answer:", response_text)
        return {"messages": [AIMessage(content=response_text)]}
    else:
        raise Exception(f"API call failed: {response.text}")

Now lets create a decision-making function that checks if we need to use any tools based on the message content. Lets look at the last message and check if it matches our tool pattern (which looks for “Action:” at the start). If it matches, signal that we should retrieve information using our tools. If it doesn’t match, signal that we should end the process.

In [30]:
tools_pattern = re.compile(r"Action: .*")

def custom_tools_condition(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    content = last_message.content

    print("Checking tools condition:", content)
    if tools_pattern.match(content):
        print("Moving to retrieve...")
        return "tools"
    print("Moving to END...")
    return END

Graph set-up

In [31]:
workflow = StateGraph(AgentState)

workflow.add_node("traffic_router_agent", traffice_router_agent)
retrieve_node = ToolNode(tools)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("rewrite", rewrite)
workflow.add_node("generate", generate)

workflow.add_edge(START, "traffic_router_agent")

# If the agent calls a tool, proceed to retrieve; otherwise, go to END
workflow.add_conditional_edges(
    "traffic_router_agent",
    custom_tools_condition,
    {
        "tools": "retrieve",
        END: END
    }
)

# After retrieve, determine whether to generate or rewrite
workflow.add_conditional_edges("retrieve", simple_grade_documents)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "traffic_router_agent")

app = workflow.compile()

In [32]:
def process_question(user_question, config):
    """Process user question through the workflow"""
    events = []
    for event in app.stream({"messages":[("user", user_question)]}, config):
        events.append(event)
    return events

In [34]:
q = "What is the latest advancement in AI research?"
events = process_question(q, {"configurable":{"thread_id":"1"}})

---CALL AGENT---


/Users/deanorenstein/opt/anaconda3/envs/oai/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.deepseek.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Raw response: SEARCH_RESEARCH: latest advancements in AI research 2023
Checking tools condition: Action: research_db_tool
{"query": "latest advancements in AI research 2023"}

Results: [Document(metadata={}, page_content='Latest Trends in Machine Learning Methods Using Quantum Computing'), Document(metadata={}, page_content='Research Report: Results of a New AI Model Improving Image Recognition Accuracy to 98%'), Document(metadata={}, page_content='Project A: UI Design Completed, API Integration in Progress'), Document(metadata={}, page_content='Academic Paper Summary: Why Transformers Became the Mainstream Architecture in Natural Language')]
Moving to retrieve...
Evaluating message: Action: research_db_tool
{"query": "latest advancements in AI research 2023"}

Results: [Document(metadata={}, page_content='Latest Trends in Machine Learning Methods Using Quantum Computing'), Document(metadata={}, page_content='Research Report: Results of a New AI Model Improving Image Recognition Accura

/Users/deanorenstein/opt/anaconda3/envs/oai/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.deepseek.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Final Answer: Based on the provided research documents, the latest advancements in AI research can be summarized as follows:

1. **Quantum Computing in Machine Learning**:  
   One of the latest trends involves the integration of quantum computing with machine learning methods. This approach aims to leverage the computational power of quantum systems to solve complex problems more efficiently than classical computing methods. Quantum computing is expected to enhance the speed and scalability of AI models, particularly in areas requiring massive data processing and optimization.

2. **Improved Image Recognition Accuracy**:  
   A significant breakthrough has been achieved in image recognition, with a new AI model achieving **98% accuracy**. This advancement demonstrates the potential for AI to perform highly precise visual tasks, which could have applications in fields such as medical imaging, autonomous vehicles, and security systems. The research highlights the importance of refining 